$\newcommand{\re}{\mathbb{R}}$
# Gaussian elimination
Given $A \in \re^{m \times m}$, we want to write it as

$$
A = L U
$$

where $L$ is lower triangular and $U$ is upper triangular.

In [15]:
%config InlineBackend.figure_format = 'svg'
import numpy as np
import matplotlib.pyplot as plt

## Gaussian elimination without pivoting

We now implement Algorithm 20.1.

In [16]:
def LU(A):
    m = A.shape[0]
    U = A.copy(); L = np.eye(m)
    for k in range(m-1):
        for j in range(k+1,m):
            L[j,k] = U[j,k] / U[k,k]
            U[j,k:] = U[j,k:] - L[j,k] * U[k,k:]
    return L, U

Generate a random matrix and compute the decomposition

In [17]:
m = 10
A = 2 * np.random.rand(m,m) - 1
L, U = LU(A)

Test that $A = L U$ is satisfied, compute $\| A - L U \|$

In [18]:
np.linalg.norm(A - L @ U)

1.4721408350624578e-13

## Solution using LU decomposition

This performs performs solution of
$$
LUx=b
$$
in two steps. In first step, solve
$$
Ly = b
$$
using
$$
y_i = \frac{1}{L_{ii}} \left[b_i - \sum_{j=0}^{i-1} L_{ij} y_j\right], \qquad i=0,1,\ldots,m-1
$$
Note that $L_{ii}=1$, so we can drop the division step. In second step, solve
$$
Ux = y
$$
using
$$
x_i = \frac{1}{U_{ii}}\left[y_i - \sum_{j=i+1}^{m-1} U_{ij} x_j \right], \qquad i=m-1,m-2,\ldots,0
$$

In [19]:
def LUSolve(L,U,b):
    m = L.shape[0]
    # solve Ly = b
    y = np.empty_like(b)
    for i in range(m):
        y[i] = b[i] - L[i,0:i].dot(y[0:i])
    # solve Ux = y
    x = np.empty_like(b)
    for i in range(m-1,-1,-1):
        x[i] = (y[i] - U[i,i+1:m].dot(x[i+1:m]))/U[i,i]
    return x

Create a random right hand side vector $b$ and solve $A x = b$.

In [20]:
b = np.random.rand(m)
x = LUSolve(L,U,b)
print(np.linalg.norm(A@x - b))

7.732516367787124e-13


## Example

Solve $A x = b$ with 

$$
A = \begin{bmatrix}
10^{-20} & 1 \\
1 & 1 \end{bmatrix}, \qquad
b = \begin{bmatrix}
1 \\ 0 \end{bmatrix}
$$

whose exact solution is

$$
x \approx \begin{bmatrix} -1 \\ 1 \end{bmatrix}
$$

In [21]:
A = np.array([[1.0e-20,  1.0],
              [1.0,      1.0]])
L, U = LU(A)
b = np.array([1.0, 0.0])
x = LUSolve(L,U,b)
print('x    = ', x)
print('Ax-b = ', A@x-b)

x    =  [0. 1.]
Ax-b =  [0. 1.]


If we interchange the two equations, then

In [22]:
A = np.array([[1.0,      1.0],
              [1.0e-20,  1.0]])
L, U = LU(A)
b = np.array([0.0, 1.0])
x = LUSolve(L,U,b)
print('x    = ', x)
print('Ax-b = ', A@x-b)

x    =  [-1.  1.]
Ax-b =  [0. 0.]


## Gaussian elimination with pivoting